In [1]:
using MHDFlows,PyPlot,CUDA,BenchmarkTools
using LinearAlgebra: mul!, ldiv!

┌ Info: FourierFlows will use 8 threads
└ @ FourierFlows /home/doraho/.julia/packages/FourierFlows/2BZya/src/FourierFlows.jl:123


# HD

In [2]:
function ProblemGeneratorTG!(prob,L0,U0;N = prob.grid.nx)
  R     = 0;

  # Output Setting  
  kx,ky,kz = fill(0.0,N,N,N),fill(0.0,N,N,N),fill(0.0,N,N,N);
  
  l = 2*π/L0;
    
  for k ∈ 1:N, j ∈ 1:N, i ∈ 1:N
    kx[i,j,k] = l*prob.grid.x[i];
    ky[i,j,k] = l*prob.grid.y[j];
    kz[i,j,k] = l*prob.grid.z[k];
  end
    
  pfactor =  4/3*sqrt(2/3);
    
  θ1 = asin(-(√(3)+R)/2/√(1+R^2));
  Φ1 = asin((√(3)-R)/2/√(1+R^2));
  ϕ1 = asin(1/(1+R^2));
    
  ux = @. (sin(kx+θ1)*cos(ky+Φ1)*sin(kz+ϕ1) - cos(kz+θ1)*sin(kx+Φ1)*sin(ky+ϕ1));
  
  uy = @. (sin(ky+θ1)*cos(kz+Φ1)*sin(kx+ϕ1) - cos(kx+θ1)*sin(ky+Φ1)*sin(kz+ϕ1));
  
  uz = @. (sin(kz+θ1)*cos(kx+Φ1)*sin(ky+ϕ1) - cos(ky+θ1)*sin(kz+Φ1)*sin(kx+ϕ1));


  copyto!(prob.vars.ux, U0*pfactor*ux);
  copyto!(prob.vars.uy, U0*pfactor*uy);
  copyto!(prob.vars.uz, U0*pfactor*uz);

  #Update V  Fourier Conponment
  uxh = prob.sol[:, :, :, prob.params.ux_ind];
  uyh = prob.sol[:, :, :, prob.params.uy_ind];
  uzh = prob.sol[:, :, :, prob.params.uz_ind];

  mul!(uxh, prob.grid.rfftplan, prob.vars.ux);   
  mul!(uyh, prob.grid.rfftplan, prob.vars.uy);
  mul!(uzh, prob.grid.rfftplan, prob.vars.uz);
    
  prob.sol[:, :, :, prob.params.ux_ind] .= uxh;
  prob.sol[:, :, :, prob.params.uy_ind] .= uyh;
  prob.sol[:, :, :, prob.params.uz_ind] .= uzh;
    
  return nothing
end

ProblemGeneratorTG! (generic function with 1 method)

In [3]:
#Simulation's parameters
N = 32;
Lx = 2π;
Re = 50;
U0 = 6.5
ν = 2*π*U0/Re;
dt = 1/500;

# Testing the problem
# Declare the problem on GPU
CPUprob  = Problem(CPU();nx = N,
                         Lx = Lx,
                          ν = ν,
                         nν = 1,
           # Timestepper and equation options
                         dt = dt,
                    stepper = "RK4",
           # Float type and dealiasing
                          T = Float32);
CPUprob

 # Set up the initial condition
ProblemGeneratorTG!(CPUprob,2π,U0);
t = @time begin TimeIntegrator!(CPUprob,1.0,20;
                           usr_dt = dt,
                            diags = [],
                      loop_number = 100,
                             save = false,
                         save_loc = "",
                         filename = "",
                          dump_dt = 0); end

  5.389445 seconds (12.02 M allocations: 722.288 MiB, 3.34% gc time, 43.48% compilation time)


In [4]:
#Simulation's parameters
N = 64;
Lx = 2π;
Re = 50;
U0 = 6.5
ν = 2*π*U0/Re;
dt = 1/500;

# Testing the problem
# Declare the problem on GPU
CPUprob  = Problem(CPU();nx = N,
                         Lx = Lx,
                          ν = ν,
                         nν = 1,
           # Timestepper and equation options
                         dt = dt,
                    stepper = "RK4",
           # Float type and dealiasing
                          T = Float32);
CPUprob

 # Set up the initial condition
ProblemGeneratorTG!(CPUprob,2π,U0);
t = @time begin TimeIntegrator!(CPUprob,1.0,20;
                           usr_dt = dt,
                            diags = [],
                      loop_number = 100,
                             save = false,
                         save_loc = "",
                         filename = "",
                          dump_dt = 0); end

  4.322912 seconds (1.25 M allocations: 739.926 MiB, 6.07% gc time, 1.01% compilation time)


In [ ]:
#Simulation's parameters
N = 128;
Lx = 2π;
Re = 50;
U0 = 6.5
ν = 2*π*U0/Re;
dt = 1/500;

# Testing the problem
# Declare the problem on GPU
CPUprob  = Problem(CPU();nx = N,
                         Lx = Lx,
                          ν = ν,
                         nν = 1,
           # Timestepper and equation options
                         dt = dt,
                    stepper = "RK4",
           # Float type and dealiasing
                          T = Float32);
CPUprob

 # Set up the initial condition
ProblemGeneratorTG!(CPUprob,2π,U0);
t = @time begin TimeIntegrator!(CPUprob,1.0,20;
                           usr_dt = dt,
                            diags = [],
                      loop_number = 100,
                             save = false,
                         save_loc = "",
                         filename = "",
                          dump_dt = 0); end

In [ ]:
#Simulation's parameters
N = 256;
Lx = 2π;
Re = 50;
U0 = 6.5
ν = 2*π*U0/Re;
dt = 1/500;

# Testing the problem
# Declare the problem on GPU
CPUprob  = Problem(CPU();nx = N,
                         Lx = Lx,
                          ν = ν,
                         nν = 1,
           # Timestepper and equation options
                         dt = dt,
                    stepper = "RK4",
           # Float type and dealiasing
                          T = Float32);
CPUprob

 # Set up the initial condition
ProblemGeneratorTG!(CPUprob,2π,U0);
t = @time begin TimeIntegrator!(CPUprob,1.0,20;
                           usr_dt = dt,
                            diags = [],
                      loop_number = 100,
                             save = false,
                         save_loc = "",
                         filename = "",
                          dump_dt = 0); end